# side quest

In [ ]:
# mode 1:  inline / chained / affecting
# surface transform = input.transforms - anchor + scale
# output transform = input.transforms - first anchor + second anchor

# mode 2: offline / free / independent
# surface transform = input.transforms + local transform - anchor + scale
# output transform = input.transforms

# how to support absolute position on chain?

# RS(X - A) + T
# surface transform(X) = CSX - A
# surface transform = anchor1 + scale + chain
# output transform = chain + anchor1 + anchor2